# LAB - Semana 13 - Sesión 47

# Índice de contenidos
1. Antes de empezar:

2. Reto 1 - Importar y describir el conjunto de datos

    2.0.0.1 Explore el conjunto de datos con técnicas matemáticas y de visualización. ¿Qué encuentra?

3. Reto 2 - Limpieza y transformación de datos

4. Reto 3 - Preprocesamiento de datos

    4.0.0.1 Utilizaremos el StandardScaler de sklearn.preprocessing y escalaremos nuestros datos. Lea más sobre StandardScaler aquí.

5. Reto 4 - Agrupación de datos con K-Means

6. Reto 5 - Agrupación de datos con DBSCAN

7. Reto 6 - Comparar K-Means con DBSCAN

8. Reto adicional 2 - Cambiar el número de clusters de K-Means

9. Bonus Challenge 3 - Cambiar DBSCAN eps y min_samples

# Antes de empezar:
- Lee el archivo README.md
- Comenta todo lo que puedas y utiliza los recursos del archivo README.md
- ¡Feliz aprendizaje!

In [ ]:
# Import your libraries:

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings                                              
from sklearn.exceptions import DataConversionWarning          
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# Desafío 1 - Importar y describir el conjunto de datos

En este laboratorio, utilizaremos un conjunto de datos que contiene información sobre las preferencias de los clientes. Analizaremos cuánto gasta cada cliente en un año en cada subcategoría de la tienda de comestibles e intentaremos encontrar similitudes mediante la agrupación.

El origen del conjunto de datos es [aquí](https://archive.ics.uci.edu/ml/datasets/wholesale+customers).

In [ ]:
# loading the data: Wholesale customers data
wholesale = pd.read_csv(r"C:\Users\tatan\Desktop\repo_course\Week13_Session47_lab-unsupervised-learning-es\data\Wholesale_customers_data.csv")


In [ ]:

wholesale.head()

In [ ]:
# Obtener información sobre el conjunto de datos
wholesale.info()

#### Explora el conjunto de datos con técnicas matemáticas y de visualización. ¿Qué encuentras?

Lista de comprobación:

* ¿Qué significa cada columna?
* ¿Hay datos categóricos que convertir?
* ¿Hay que eliminar datos que faltan?
* Colinealidad de columnas: ¿hay correlaciones altas?
* Estadísticas descriptivas: ¿hay que eliminar algún valor atípico?
* Distribución de los datos por columnas: ¿está sesgada la distribución?
* Etc.

Información adicional: Hace más de un siglo, un economista italiano llamado Vilfredo Pareto descubrió que aproximadamente el 20% de los clientes representan el 80% de las ventas minoristas típicas. Esto se denomina [principio de Pareto](https://en.wikipedia.org/wiki/Pareto_principle). Compruebe si este conjunto de datos presenta esta característica.

In [ ]:
# Your code here:
x = wholesale

# Análisis de calidad de la base de datos
print("TOTAL NUMBER OF ROWS: \n",len(x),"\n")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -\n")
print("NUMBER OF UNIQUE VALUES:\n",x.nunique(),"\n")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - \n")
print("NUMBER OF UNIQUE VALUES DIVIDED BY TOTAL VALUES:\n",round((x.nunique()/len(x)*100),2),"%\n")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - \n")
print("% OF NAN VALUES IS: \n",round(100*(x.isnull().sum() / len(x)),2),"%\n")
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - \n")

print(x.describe())

# Seleccionar solo las columnas numéricas
numeric_columns = x.select_dtypes(include='number')

# Crear un boxplot para cada columna numérica
plt.figure(figsize=(12, 8))
sns.boxplot(data=numeric_columns, palette='pastel')

# Personalizar el gráfico
plt.title("Boxplot de las columnas numéricas")
plt.ylabel("Valores")
plt.xlabel("Columnas")
plt.xticks(rotation=45)  # Rotar los nombres de las columnas para facilitar lectura
plt.show()

# Reto 2 - Limpieza y transformación de datos

Si tu conclusión del reto anterior es que los datos necesitan limpieza/transformación, hazlo en las celdas de abajo. Sin embargo, si su conclusión es que los datos no necesitan ser limpiados o transformados, no dudes en saltarte este reto. Si optas por esta última opción, explica los motivos.

In [ ]:
# Your code here
# Eliminar Outliers
def remove_outliers(x, column):
     Q1 = x[column].quantile(0.25)  # Primer cuartil
     Q3 = x[column].quantile(0.75)  # Tercer cuartil
     IQR = Q3 - Q1  # Rango intercuartil
     lower_bound = Q1 - 1.5 * IQR  # Límite inferior
     upper_bound = Q3 + 1.5 * IQR  # Límite superior

# Filtrar datos dentro de los límites
     return x[(x[column] >= lower_bound) & (x[column] <= upper_bound)]

In [ ]:
# Aplicar una transformación logarítmica
def log_transfom_clean_(x):
    if np.isfinite(x) and x!=0:
        return np.log(x)
    else:
        return np.NAN
# Limpiar outliers para cada columna

for column in x.columns:
    x_transformed = remove_outliers(x, column)
x_transformed = x.map(log_transfom_clean_)
print("DataFrame sin outliers:\n", x_transformed)

# Seleccionar solo las columnas numéricas
numeric_columns = x_transformed.select_dtypes(include='number')

# Crear un boxplot para cada columna numérica
plt.figure(figsize=(12, 8))
sns.boxplot(data=numeric_columns, palette='pastel')

# Personalizar el gráfico
plt.title("Boxplot de las columnas numéricas")
plt.ylabel("Valores")
plt.xlabel("Columnas")
plt.xticks(rotation=45)  # Rotar los nombres de las columnas para facilitar lectura
plt.show()

**Tus observaciones aquí**

+ Con la transformación logarítmica, la dispersión se reduce y los outliers se concentran en la parte inferior del boxplot. Aunque la columna 'Fresh' sigue teniendo muchos outliers, podrían ser útiles para distinguir patrones. En contraste, con la transformación de raíz cuadrada, los outliers se desplazan hacia la parte superior.

# Reto 3 - Preprocesamiento de datos

Uno de los problemas del conjunto de datos es que los rangos de valores son notablemente diferentes en las distintas categorías (por ejemplo, `Fresh` y `Grocery` en comparación con `Detergents_Paper` y `Delicassen`). Si hiciste esta observación en el primer reto, ¡has hecho un gran trabajo! Esto significa que no sólo has completado las preguntas de bonificación en el anterior laboratorio de Aprendizaje Supervisado, sino que también has investigado en profundidad sobre [*feature scaling*](https://en.wikipedia.org/wiki/Feature_scaling). ¡Sigue trabajando así de bien!

Diversos rangos de valores en diferentes características podrían causar problemas en nuestra agrupación. La forma de reducir el problema es mediante el escalado de características. Volveremos a utilizar esta técnica con este conjunto de datos.

#### Utilizaremos el `StandardScaler` de `sklearn.preprocessing` y escalaremos nuestros datos. Lee más sobre `StandardScaler` [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

*Después de escalar tus datos, asigna los datos transformados a una nueva variable `customers_scale`.

In [ ]:
# Your import here:

from sklearn.preprocessing import StandardScaler

# Your code here:
scaler = StandardScaler()
print(scaler.fit_transform(x_transformed))
customers_scale = scaler.transform(x_transformed)

# Reto 4 - Agrupación de datos con K-Means

Ahora vamos a agrupar los datos con K-Means primero. Inicia el modelo K-Means, luego ajusta tus datos escalados. En los datos devueltos por el método `.fit`, hay un atributo llamado `labels_` que es el número de cluster asignado a cada registro de datos. Lo que puede hacer es asignar estas etiquetas de nuevo a `customers` en una nueva columna llamada `customers['labels']`. Entonces verá los resultados de cluster de los datos originales.

In [ ]:
from sklearn.cluster import KMeans

# Your code here:


### Viendo el elbow pododríamos escoger 2 como el número de clusters correctos

In [ ]:
kmeans_2 = KMeans(n_clusters=2).fit(customers_scale)

labels = kmeans_2.predict(customers_scale)

clusters = kmeans_2.labels_.tolist()

In [ ]:
x_transformed['Label'] = clusters

Cuenta los valores en `labels`.

In [ ]:
# Your code here:
print(x_transformed['Label'].value_counts())

# Reto 5 - Clustering de datos con DBSCAN

Ahora vamos a agrupar los datos utilizando DBSCAN. Utiliza `DBSCAN(eps=0.5)` para iniciar el modelo y, a continuación, ajusta los datos escalados. En los datos devueltos por el método `.fit`, asigna las `labels_` de nuevo a `customers['labels_DBSCAN']`. Ahora tus datos originales tienen dos etiquetas, una de K-Means y la otra de DBSCAN.

In [ ]:
from sklearn.cluster import DBSCAN 

# Your code here


Cuenta los valores en `labels_DBSCAN`.

In [ ]:
# Your code here
X_scaled = x_transformed.drop(columns=['Label'])

dbscan = DBSCAN(eps=2, min_samples=5)
clusters = dbscan.fit_predict(X_scaled)

x_transformed['labels_DBSCAN'] = clusters

In [ ]:

print(x_transformed['labels_DBSCAN'].value_counts())

# Reto 6 - Comparar K-Means con DBSCAN

Ahora queremos comparar visualmente cómo K-Means y DBSCAN han agrupado nuestros datos. Crearemos gráficos de dispersión para varias columnas. Para cada uno de los siguientes pares de columnas, traza un gráfico de dispersión utilizando `labels` y otro utilizando `labels_DBSCAN`. Ponlos uno al lado del otro para compararlos. ¿Qué algoritmo de agrupación tiene más sentido?

Columnas a visualizar:

* `Detergents_Paper` as X and `Milk` as y
* `Grocery` as X and `Fresh` as y
* `Frozen` as X and `Delicassen` as y

Visualice `Detergentes_Papel` como X y `Leche` como Y mediante `labels` y `labels_DBSCAN` respectivamente

In [ ]:
def plot(x,y,hue):
    sns.scatterplot(x=x, 
                    y=y,
                    hue=hue)
    plt.title('Detergents Paper vs Milk ')
    return plt.show();

In [ ]:
# Your code here:
plot(x_transformed.Detergents_Paper,x_transformed.Milk, x_transformed.Label)
plot(x_transformed.Detergents_Paper,x_transformed.Milk, x_transformed.labels_DBSCAN)

Visualice `Grocery` como X y `Fresh` como Y mediante `labels` y `labels_DBSCAN` respectivamente

In [ ]:
# Your code here:
plot(x_transformed.Grocery,x_transformed.Fresh, x_transformed.Label)
plot(x_transformed.Grocery,x_transformed.Fresh, x_transformed.labels_DBSCAN)

Visualice `Frozen` como X y `Delicassen` como Y mediante `labels` y `labels_DBSCAN` respectivamente

In [ ]:
# Your code here:
plot(x_transformed.Frozen,x_transformed.Delicassen, x_transformed.Label)
plot(x_transformed.Frozen,x_transformed.Delicassen, x_transformed.labels_DBSCAN)

Vamos a utilizar un groupby para ver cómo la media difiere entre los grupos. Agrupamos `customers` por `labels` y `labels_DBSCAN` respectivamente y calculamos las medias de todas las columnas.

In [ ]:
# Your code here:
x_transformed.groupby(['Label']).mean()

¿Qué algoritmo funciona mejor?

**Tus observaciones aquí**

Las gráficas muestran que K-Means identifica 2 clusters claramente, aunque cercanos, mientras que DBSCAN mezcla hasta 3 sin diferenciarlos bien. Por ahora, K-Means parece más efectivo, aunque los datos podrían mejorarse para obtener clusters más definidos.

# Bonus Challenge 2 - Cambiar el número de clusters de K-Means

Como hemos mencionado antes, no tenemos que preocuparnos por el número de clusters con DBSCAN porque lo decide automáticamente en función de los parámetros que le enviemos. Pero con K-Means, tenemos que suministrar el parámetro `n_clusters` (si no se suministra `n_clusters`, el algoritmo utilizará `8` por defecto). Debe saber que el número óptimo de clusters varía en función del conjunto de datos. K-Means puede funcionar mal si se utiliza un número incorrecto de clusters.

En el aprendizaje automático avanzado, los científicos de datos prueban diferentes números de clusters y evalúan los resultados con medidas estadísticas (leer [aquí](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation)). Hoy no vamos a utilizar medidas estadísticas, sino nuestros ojos. En las celdas de abajo, experimenta con distintos números de conglomerados y visualízalos con gráficos de dispersión. ¿Qué número de clusters parece funcionar mejor para K-Means?

In [ ]:
# Your code here
customers_scale

In [ ]:
xKM3 = x_transformed.drop(columns='labels_DBSCAN')

kmeans_4 = KMeans(n_clusters=3).fit(customers_scale)

labels = kmeans_4.predict(customers_scale)

clusters3 = kmeans_4.labels_.tolist()

xKM3['Label'] = clusters3

plot(xKM3.Detergents_Paper,xKM3.Milk, xKM3.Label)
plot(xKM3.Grocery,xKM3.Fresh, xKM3.Label)
plot(xKM3.Frozen,xKM3.Delicassen, xKM3.Label)

xKM3

**Tus observaciones aquí**

* Los gráficos de K-Means sugieren un límite de 3 clusters, aunque con mucho ruido.

# Bonus Challenge 3 - Cambiar `eps` y `min_samples` de DBSCAN

Experimenta cambiando los parámetros `eps` y `min_samples` de DBSCAN. Mira cómo difieren los resultados con la visualización de gráficos de dispersión.

In [ ]:
# Your code here
X_scaled = x_transformed.drop(columns=['Label'])

dbscan = DBSCAN(eps=0.5, min_samples=10)
clusters = dbscan.fit_predict(X_scaled)

x_transformed['labels_DBSCAN'] = clusters
plot(x_transformed.Detergents_Paper,x_transformed.Milk, x_transformed.labels_DBSCAN)

**Tus observaciones aquí**

    + No he logrado que el DBSCAN funcione adecuadamente con las imágenes disponibles, incluso después de realizar ajustes.
    
